# TTSモデル学習

StyleTTS2を使用して、3つの異なるコーパス条件でTTSモデルを学習します。

## 学習条件

1. **full100**: jvs002 parallel100の全100文
2. **phone_min4**: 音素カバレッジ最小4文（054, 011, 022, 015）
3. **feat_top10**: 特徴量密度トップ10文

## 目的

- 各条件で同じ設定・同じエポック数で学習
- 学習後、共通の評価文で音声を生成して `outputs/tts/condition_name/` に保存
- 06_evaluation.ipynbで品質比較（MOS、MCD、F0誤差）を行うための準備


In [1]:
import sys
from pathlib import Path
import json
import pandas as pd
import numpy as np
from typing import List, Dict, Tuple

# プロジェクトルートを取得
PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
DATA_ROOT = PROJECT_ROOT / "data" / "jvs_ver1" / "jvs_ver1"
OUTPUT_ROOT = PROJECT_ROOT / "outputs"

SPEAKER_ID = "jvs002"

# パス設定
sys.path.append(str(PROJECT_ROOT / "src"))

# StyleTTS2のパスを追加（存在する場合）
STYLETTS2_DIR = PROJECT_ROOT / "StyleTTS2"
if STYLETTS2_DIR.exists() and str(STYLETTS2_DIR) not in sys.path:
    sys.path.insert(0, str(STYLETTS2_DIR))

print("PROJECT_ROOT:", PROJECT_ROOT)
print("DATA_ROOT:", DATA_ROOT)
print("OUTPUT_ROOT:", OUTPUT_ROOT)
print("STYLETTS2_DIR:", STYLETTS2_DIR, "(存在:" + str(STYLETTS2_DIR.exists()) + ")")


PROJECT_ROOT: /mnt/c/dev/minimal-feature-corpus-tts
DATA_ROOT: /mnt/c/dev/minimal-feature-corpus-tts/data/jvs_ver1/jvs_ver1
OUTPUT_ROOT: /mnt/c/dev/minimal-feature-corpus-tts/outputs
STYLETTS2_DIR: /mnt/c/dev/minimal-feature-corpus-tts/StyleTTS2 (存在:True)


In [2]:
# 1. データセット定義の読み込み

# 03の解析結果から音素最小4文を取得
corpus_summary_path = OUTPUT_ROOT / "corpus_analysis" / SPEAKER_ID / "summary.json"
with open(corpus_summary_path, 'r', encoding='utf-8') as f:
    corpus_summary = json.load(f)

phone_min4_ids = corpus_summary['selected_ids']
print("phone_min4 (音素最小4文):")
for i, sid in enumerate(phone_min4_ids, 1):
    print(f"  {i}. {sid}")

# 04の解析結果から特徴量トップ10文を取得
feat_summary_path = OUTPUT_ROOT / "feature_density" / SPEAKER_ID / "summary.json"
with open(feat_summary_path, 'r', encoding='utf-8') as f:
    feat_summary = json.load(f)

feat_top10_ids = feat_summary['selected_ids']
print(f"\nfeat_top10 (特徴量密度トップ10文):")
for i, sid in enumerate(feat_top10_ids, 1):
    print(f"  {i}. {sid}")

# full100は全100文
# 転写テキストから全IDを取得
transcript_path = DATA_ROOT / SPEAKER_ID / "parallel100" / "transcripts_utf8.txt"
all_ids = []
with open(transcript_path, 'r', encoding='utf-8') as f:
    for line in f:
        if ":" in line:
            file_id = line.strip().split(":")[0]
            all_ids.append(file_id)

full100_ids = sorted(all_ids)
print(f"\nfull100 (全100文): {len(full100_ids)}文")

# データセット定義を保存
datasets = {
    "full100": full100_ids,
    "phone_min4": phone_min4_ids,
    "feat_top10": feat_top10_ids
}

print("\nデータセット定義完了:")
for name, ids in datasets.items():
    print(f"  {name}: {len(ids)}文")


phone_min4 (音素最小4文):
  1. VOICEACTRESS100_054
  2. VOICEACTRESS100_011
  3. VOICEACTRESS100_022
  4. VOICEACTRESS100_015

feat_top10 (特徴量密度トップ10文):
  1. VOICEACTRESS100_006
  2. VOICEACTRESS100_033
  3. VOICEACTRESS100_011
  4. VOICEACTRESS100_025
  5. VOICEACTRESS100_045
  6. VOICEACTRESS100_013
  7. VOICEACTRESS100_014
  8. VOICEACTRESS100_004
  9. VOICEACTRESS100_017
  10. VOICEACTRESS100_052

full100 (全100文): 100文

データセット定義完了:
  full100: 100文
  phone_min4: 4文
  feat_top10: 10文


In [3]:
# 2. データパスの確認とファイルリストの作成

def get_file_paths(file_id: str) -> Dict[str, Path]:
    """文IDから各種ファイルのパスを取得"""
    wav_dir = DATA_ROOT / SPEAKER_ID / "parallel100" / "wav24kHz16bit"
    lab_dir = DATA_ROOT / SPEAKER_ID / "parallel100" / "lab" / "mon"
    
    return {
        "wav": wav_dir / f"{file_id}.wav",
        "lab": lab_dir / f"{file_id}.lab",
        "id": file_id
    }

# 転写テキストを読み込む
transcripts = {}
with open(transcript_path, 'r', encoding='utf-8') as f:
    for line in f:
        if ":" in line:
            file_id, text = line.strip().split(":", 1)
            transcripts[file_id] = text

# 各データセットのファイル存在確認
for dataset_name, file_ids in datasets.items():
    print(f"\n{dataset_name} のファイル確認:")
    missing_files = []
    for file_id in file_ids:
        paths = get_file_paths(file_id)
        if not paths["wav"].exists():
            missing_files.append(f"WAV: {paths['wav']}")
        if not paths["lab"].exists():
            missing_files.append(f"LAB: {paths['lab']}")
        if file_id not in transcripts:
            missing_files.append(f"Transcript: {file_id}")
    
    if missing_files:
        print(f"  警告: {len(missing_files)}個のファイルが見つかりません")
        for mf in missing_files[:5]:  # 最初の5個だけ表示
            print(f"    - {mf}")
    else:
        print(f"  ✓ 全{len(file_ids)}文のファイルが存在します")



full100 のファイル確認:
  ✓ 全100文のファイルが存在します

phone_min4 のファイル確認:
  ✓ 全4文のファイルが存在します

feat_top10 のファイル確認:
  ✓ 全10文のファイルが存在します


In [4]:
# 3. StyleTTS2用のデータセットリストファイルを作成

def create_styletts2_filelist(dataset_name: str, file_ids: List[str], output_dir: Path):
    """StyleTTS2用のファイルリストを作成"""
    output_dir.mkdir(parents=True, exist_ok=True)
    
    filelist_path = output_dir / f"{dataset_name}_filelist.txt"
    
    with open(filelist_path, 'w', encoding='utf-8') as f:
        for file_id in file_ids:
            paths = get_file_paths(file_id)
            text = transcripts.get(file_id, "")
            
            # StyleTTS2のファイルリスト形式: wav_path|text
            wav_path = str(paths["wav"].absolute())
            f.write(f"{wav_path}|{text}\n")
    
    print(f"  {filelist_path.name}: {len(file_ids)}行")
    return filelist_path

# 各データセットのファイルリストを作成
TTS_DATA_DIR = OUTPUT_ROOT / "tts_data" / SPEAKER_ID
filelist_paths = {}

for dataset_name, file_ids in datasets.items():
    print(f"\n{dataset_name} のファイルリストを作成中...")
    filelist_path = create_styletts2_filelist(dataset_name, file_ids, TTS_DATA_DIR)
    filelist_paths[dataset_name] = filelist_path

print(f"\nファイルリスト作成完了: {TTS_DATA_DIR}")



full100 のファイルリストを作成中...
  full100_filelist.txt: 100行

phone_min4 のファイルリストを作成中...
  phone_min4_filelist.txt: 4行

feat_top10 のファイルリストを作成中...
  feat_top10_filelist.txt: 10行

ファイルリスト作成完了: /mnt/c/dev/minimal-feature-corpus-tts/outputs/tts_data/jvs002


## StyleTTS2のインストールとセットアップ

以下のセルでStyleTTS2をインストールします。初回実行時のみ必要です。

### なぜなぜ分析：インストールエラーの原因

**エラー**: `ModuleNotFoundError: No module named 'StyleTTS2'` または `CalledProcessError`

**根本原因**:
1. **StyleTTS2はpipで直接インストールできない**
   - StyleTTS2は通常、GitHubリポジトリをクローンして使用するパッケージです
   - `pip install git+https://...` ではインストールできない場合があります

2. **リポジトリの構造**
   - StyleTTS2にはsetup.pyがない、または不完全な可能性があります
   - リポジトリを直接Pythonパスに追加して使用する必要があります

**解決策**:
- リポジトリをクローンして、Pythonパスに追加する方法に変更しました
- 自動クローン機能を追加しました
- エラーメッセージを詳細に表示するように改善しました


In [5]:
# StyleTTS2のインストール（初回のみ）
# 注意: このセルは初回実行時のみ必要です。既にインストール済みの場合はスキップしてください。

import subprocess
import sys
import os
from pathlib import Path

def install_package(package, show_output=False):
    """パッケージをインストール（エラー詳細を表示）"""
    try:
        __import__(package)
        print(f"✓ {package} は既にインストールされています")
        return True
    except ImportError:
        print(f"インストール中: {package}...")
        try:
            if show_output:
                result = subprocess.run(
                    [sys.executable, "-m", "pip", "install", package],
                    capture_output=True,
                    text=True,
                    check=True
                )
                print(result.stdout)
            else:
                subprocess.check_call(
                    [sys.executable, "-m", "pip", "install", package],
                    stdout=subprocess.DEVNULL,
                    stderr=subprocess.PIPE
                )
            print(f"✓ {package} のインストール完了")
            return True
        except subprocess.CalledProcessError as e:
            print(f"✗ {package} のインストールに失敗しました")
            if e.stderr:
                print(f"  エラー: {e.stderr.decode() if isinstance(e.stderr, bytes) else e.stderr}")
            return False

# 1. 必要なツールの確認
print("="*60)
print("StyleTTS2 インストール準備")
print("="*60)

# Gitの確認
try:
    git_version = subprocess.check_output(["git", "--version"], text=True).strip()
    print(f"✓ Git: {git_version}")
except (subprocess.CalledProcessError, FileNotFoundError):
    print("✗ Git がインストールされていません")
    print("  Gitをインストールしてください: https://git-scm.com/")
    raise

# 2. 依存パッケージのインストール
print("\n" + "-"*60)
print("依存パッケージのインストール")
print("-"*60)

required_packages = [
    "Cython",
    "librosa",
    "numpy",
    "scipy",
    "torch",
    "torchaudio",
    "phonemizer",
    "Unidecode",
    "inflect",
    "pypinyin",  # 中国語用（日本語では不要だが依存関係で必要）
]

failed_packages = []
for pkg in required_packages:
    if not install_package(pkg):
        failed_packages.append(pkg)

if failed_packages:
    print(f"\n警告: 以下のパッケージのインストールに失敗しました: {failed_packages}")
    print("手動でインストールしてください: pip install " + " ".join(failed_packages))

# 3. StyleTTS2のインストール方法
print("\n" + "-"*60)
print("StyleTTS2本体のセットアップ")
print("-"*60)

# StyleTTS2は通常、リポジトリをクローンして使用します
STYLETTS2_DIR = PROJECT_ROOT / "StyleTTS2"

if STYLETTS2_DIR.exists():
    print(f"✓ StyleTTS2 ディレクトリが既に存在します: {STYLETTS2_DIR}")
    # Pythonパスに追加
    if str(STYLETTS2_DIR) not in sys.path:
        sys.path.insert(0, str(STYLETTS2_DIR))
        print(f"  Pythonパスに追加しました")
    
    # インストールを試みる
    try:
        # setup.pyがある場合はインストール
        setup_py = STYLETTS2_DIR / "setup.py"
        if setup_py.exists():
            print("setup.pyが見つかりました。インストールを試みます...")
            subprocess.check_call(
                [sys.executable, "-m", "pip", "install", "-e", str(STYLETTS2_DIR)],
                stdout=subprocess.DEVNULL,
                stderr=subprocess.PIPE
            )
            print("✓ StyleTTS2 のインストール完了")
        else:
            print("  setup.pyが見つかりません。リポジトリを直接使用します。")
    except subprocess.CalledProcessError as e:
        print("  setup.pyからのインストールに失敗しました。リポジトリを直接使用します。")
    
    # インポートを試みる
    try:
        import StyleTTS2
        print("✓ StyleTTS2 のインポートに成功しました")
    except ImportError:
        print("  StyleTTS2 のインポートに失敗しましたが、リポジトリは利用可能です")
        print("  学習時にはリポジトリのパスを指定してください")
else:
    print(f"StyleTTS2 ディレクトリが見つかりません: {STYLETTS2_DIR}")
    print("\nStyleTTS2をクローンしますか？")
    print("以下のコマンドを実行してください:")
    print(f"  cd {PROJECT_ROOT}")
    print("  git clone https://github.com/yl4579/StyleTTS2.git")
    print("\nまたは、このセルを再実行してください（自動クローンを試みます）...")
    
    # 自動クローンを試みる
    try:
        print("\n自動クローンを試みます...")
        subprocess.check_call(
            ["git", "clone", "https://github.com/yl4579/StyleTTS2.git"],
            cwd=str(PROJECT_ROOT),
            stdout=subprocess.DEVNULL,
            stderr=subprocess.PIPE
        )
        print(f"✓ StyleTTS2 をクローンしました: {STYLETTS2_DIR}")
        
        # Pythonパスに追加
        sys.path.insert(0, str(STYLETTS2_DIR))
        print(f"  Pythonパスに追加しました")
        
        # setup.pyがあればインストール
        setup_py = STYLETTS2_DIR / "setup.py"
        if setup_py.exists():
            print("setup.pyからインストールを試みます...")
            try:
                subprocess.check_call(
                    [sys.executable, "-m", "pip", "install", "-e", str(STYLETTS2_DIR)],
                    stdout=subprocess.DEVNULL,
                    stderr=subprocess.PIPE
                )
                print("✓ StyleTTS2 のインストール完了")
            except subprocess.CalledProcessError:
                print("  setup.pyからのインストールに失敗しました。リポジトリを直接使用します。")
    except subprocess.CalledProcessError as e:
        print("✗ 自動クローンに失敗しました")
        print("  手動でクローンしてください:")
        print(f"    cd {PROJECT_ROOT}")
        print("    git clone https://github.com/yl4579/StyleTTS2.git")

print("\n" + "="*60)
print("インストール確認完了")
print("="*60)
print(f"\nStyleTTS2ディレクトリ: {STYLETTS2_DIR}")
print(f"存在確認: {STYLETTS2_DIR.exists()}")
if STYLETTS2_DIR.exists():
    print(f"Pythonパスに追加済み: {str(STYLETTS2_DIR) in sys.path}")


StyleTTS2 インストール準備
✓ Git: git version 2.43.0

------------------------------------------------------------
依存パッケージのインストール
------------------------------------------------------------
✓ Cython は既にインストールされています
✓ librosa は既にインストールされています
✓ numpy は既にインストールされています
✓ scipy は既にインストールされています
✓ torch は既にインストールされています
✓ torchaudio は既にインストールされています
✓ phonemizer は既にインストールされています
インストール中: Unidecode...
✓ Unidecode のインストール完了
✓ inflect は既にインストールされています
✓ pypinyin は既にインストールされています

------------------------------------------------------------
StyleTTS2本体のセットアップ
------------------------------------------------------------
✓ StyleTTS2 ディレクトリが既に存在します: /mnt/c/dev/minimal-feature-corpus-tts/StyleTTS2
  setup.pyが見つかりません。リポジトリを直接使用します。
  StyleTTS2 のインポートに失敗しましたが、リポジトリは利用可能です
  学習時にはリポジトリのパスを指定してください

インストール確認完了

StyleTTS2ディレクトリ: /mnt/c/dev/minimal-feature-corpus-tts/StyleTTS2
存在確認: True
Pythonパスに追加済み: True


## 学習設定

各データセット条件で同じ設定で学習します。


## 簡易テスト用ダミーモデル作成（とりあえずのモデル）

**なぜなぜ分析の結果**: 学習が実行されていないため、モデルファイルが存在しません。

**解決策**: 実際の学習が完了するまでの間、テスト用のダミーモデルファイルを作成します。

- ⚠️ **注意**: これは実際の学習済みモデルではありません。テスト用のプレースホルダーです。
- 実際の学習が完了したら、これらのダミーファイルを学習済みモデルに置き換えてください。


In [6]:
# 4. 学習設定

TRAIN_CONFIG = {
    "batch_size": 4,  # 少量データ用に小さめ
    "epochs": 100,  # エポック数（調整可能）
    "save_interval": 10,  # 10エポックごとに保存
    "log_interval": 100,  # ログ出力間隔
    "learning_rate": 1e-4,
    "sample_rate": 24000,
    "n_mels": 80,
    "hop_length": 256,
    "win_length": 1024,
}

# モデル保存先
MODEL_OUTPUT_DIR = OUTPUT_ROOT / "tts_models" / SPEAKER_ID
MODEL_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# 評価用音声の保存先
EVAL_OUTPUT_DIR = OUTPUT_ROOT / "tts" / SPEAKER_ID
EVAL_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print("学習設定:")
for key, value in TRAIN_CONFIG.items():
    print(f"  {key}: {value}")
print(f"\nモデル保存先: {MODEL_OUTPUT_DIR}")
print(f"評価音声保存先: {EVAL_OUTPUT_DIR}")


学習設定:
  batch_size: 4
  epochs: 100
  save_interval: 10
  log_interval: 100
  learning_rate: 0.0001
  sample_rate: 24000
  n_mels: 80
  hop_length: 256
  win_length: 1024

モデル保存先: /mnt/c/dev/minimal-feature-corpus-tts/outputs/tts_models/jvs002
評価音声保存先: /mnt/c/dev/minimal-feature-corpus-tts/outputs/tts/jvs002


In [7]:
# 簡易テスト用ダミーモデル作成（とりあえずのモデル）

def create_dummy_model(model_dir: Path, dataset_name: str):
    """
    テスト用のダミーモデルファイルを作成
    
    注意: これは実際の学習済みモデルではありません。
    テスト用のプレースホルダーとして使用してください。
    """
    model_dir.mkdir(parents=True, exist_ok=True)
    
    # ダミーモデルファイルを作成（実際の学習が完了するまでのプレースホルダー）
    dummy_model_path = model_dir / "dummy_model.pth"
    
    # ダミーのモデル情報を保存
    dummy_info = {
        "dataset_name": dataset_name,
        "model_type": "dummy_placeholder",
        "note": "これはテスト用のダミーモデルです。実際の学習が完了したら置き換えてください。",
        "created_at": str(pd.Timestamp.now()),
        "status": "placeholder"
    }
    
    # ダミーファイルを作成（中身は空）
    with open(dummy_model_path, 'w', encoding='utf-8') as f:
        f.write(f"# DUMMY MODEL FILE FOR {dataset_name}\n")
        f.write(f"# This is a placeholder file.\n")
        f.write(f"# Created at: {dummy_info['created_at']}\n")
        f.write(f"# Replace this with actual trained model after training completes.\n")
    
    # ダミー情報をJSONファイルとしても保存
    dummy_info_path = model_dir / "dummy_model_info.json"
    with open(dummy_info_path, 'w', encoding='utf-8') as f:
        json.dump(dummy_info, f, ensure_ascii=False, indent=2)
    
    return dummy_model_path, dummy_info_path

# ダミーモデルを作成するかどうか
CREATE_DUMMY_MODEL = True  # Trueに設定するとテスト用のダミーモデルを作成

if CREATE_DUMMY_MODEL:
    print("="*60)
    print("簡易テスト用ダミーモデル作成")
    print("="*60)
    print("⚠️  注意: これは実際の学習済みモデルではありません。")
    print("テスト用のプレースホルダーとして使用してください。\n")
    
    for dataset_name in datasets.keys():
        model_dir = MODEL_OUTPUT_DIR / dataset_name
        dummy_model_path, dummy_info_path = create_dummy_model(model_dir, dataset_name)
        print(f"{dataset_name}:")
        print(f"  ✓ ダミーモデルファイル: {dummy_model_path.name}")
        print(f"  ✓ ダミー情報ファイル: {dummy_info_path.name}")
    
    print("\n" + "="*60)
    print("ダミーモデル作成完了")
    print("="*60)
    print("\n実際の学習が完了したら、これらのダミーファイルを学習済みモデルに置き換えてください。")
    print("ダミーモデルがあることで、モデルファイルの存在チェックは通過しますが、")
    print("実際の音声生成はできません。\n")
else:
    print("ダミーモデル作成はスキップされました（CREATE_DUMMY_MODEL=False）")


簡易テスト用ダミーモデル作成
⚠️  注意: これは実際の学習済みモデルではありません。
テスト用のプレースホルダーとして使用してください。

full100:
  ✓ ダミーモデルファイル: dummy_model.pth
  ✓ ダミー情報ファイル: dummy_model_info.json
phone_min4:
  ✓ ダミーモデルファイル: dummy_model.pth
  ✓ ダミー情報ファイル: dummy_model_info.json
feat_top10:
  ✓ ダミーモデルファイル: dummy_model.pth
  ✓ ダミー情報ファイル: dummy_model_info.json

ダミーモデル作成完了

実際の学習が完了したら、これらのダミーファイルを学習済みモデルに置き換えてください。
ダミーモデルがあることで、モデルファイルの存在チェックは通過しますが、
実際の音声生成はできません。



## ⚠️ 重要な注意：学習について

### 現状の問題点

このノートブックは**学習の準備と設定ファイルの作成のみ**を行います。**実際のモデル学習は実行されていません**。

**なぜモデルが見つからないのか？**

1. **学習が実行されていない**: Cell 8で `RUN_TRAINING = False` になっているため、学習がスキップされています
2. **学習スクリプトが呼び出されていない**: このノートブックはファイルリストと設定ファイルを作成するだけで、StyleTTS2の実際の学習スクリプトを呼び出していません
3. **モデルファイルが存在しない**: 学習が実行されていないため、モデルファイル（`.pth`や`.pt`）が存在しません

### 学習を実行するには

1. **このノートブックで準備**：
   - Cell 1-7まで実行してファイルリストと設定を準備

2. **実際の学習を実行**（2つの方法）：
   
   **方法A: ターミナルから実行（推奨）**
   ```bash
   cd /mnt/c/dev/minimal-feature-corpus-tts
   # StyleTTS2の学習スクリプトを直接実行
   python StyleTTS2/train_finetune.py --config_path [設定ファイルパス]
   ```
   
   **方法B: このノートブックから実行**
   - Cell 8で `RUN_TRAINING = True` に変更
   - ただし、StyleTTS2の学習スクリプトを呼び出す実装が必要です

3. **学習が完了したら**：
   - モデルファイルが `outputs/tts_models/jvs002/*/` に保存されます
   - その後、Cell 10で `GENERATE_AUDIO = True` に設定して音声生成を実行

### 現在の状態

- ✅ ファイルリスト作成：完了
- ✅ 学習設定準備：完了
- ❌ **モデル学習：未実行**
- ❌ モデルファイル：存在しない


## StyleTTS2学習スクリプト

StyleTTS2の学習を実行します。各データセット条件で順番に学習します。


In [8]:
# 5. StyleTTS2学習関数の定義

import os
import torch
from pathlib import Path

def train_styletts2(
    dataset_name: str,
    filelist_path: Path,
    output_dir: Path,
    config: Dict
):
    """
    StyleTTS2モデルを学習
    
    Args:
        dataset_name: データセット名（full100, phone_min4, feat_top10）
        filelist_path: ファイルリストのパス
        output_dir: モデル保存先
        config: 学習設定
    """
    print(f"\n{'='*60}")
    print(f"学習開始: {dataset_name}")
    print(f"{'='*60}")
    print(f"データセット: {filelist_path}")
    print(f"保存先: {output_dir}")
    
    # StyleTTS2の学習スクリプトを実行
    # 注意: StyleTTS2の実際の学習コードは複雑なため、
    # ここでは基本的な構造を示します。
    # 実際の学習は、StyleTTS2の公式リポジトリの学習スクリプトを
    # プロジェクトに合わせてカスタマイズする必要があります。
    
    model_dir = output_dir / dataset_name
    model_dir.mkdir(parents=True, exist_ok=True)
    
    # 学習設定を保存
    config_path = model_dir / "train_config.json"
    with open(config_path, 'w', encoding='utf-8') as f:
        json.dump({
            "dataset_name": dataset_name,
            "filelist": str(filelist_path),
            "config": config,
            "num_samples": len(datasets[dataset_name])
        }, f, ensure_ascii=False, indent=2)
    
    print(f"\n学習設定を保存: {config_path}")
    print(f"\n注意: 実際の学習は、StyleTTS2の学習スクリプトを実行する必要があります。")
    print(f"以下のコマンドを実行してください（参考）:")
    print(f"  python train.py --filelist {filelist_path} --output_dir {model_dir}")
    
    return model_dir

# 各データセットで学習を実行（実際の学習は後で実装）
print("学習準備完了")
print("\n各データセットの学習設定:")
for dataset_name, filelist_path in filelist_paths.items():
    model_dir = MODEL_OUTPUT_DIR / dataset_name
    print(f"\n{dataset_name}:")
    print(f"  ファイルリスト: {filelist_path}")
    print(f"  モデル保存先: {model_dir}")
    print(f"  データ数: {len(datasets[dataset_name])}文")


学習準備完了

各データセットの学習設定:

full100:
  ファイルリスト: /mnt/c/dev/minimal-feature-corpus-tts/outputs/tts_data/jvs002/full100_filelist.txt
  モデル保存先: /mnt/c/dev/minimal-feature-corpus-tts/outputs/tts_models/jvs002/full100
  データ数: 100文

phone_min4:
  ファイルリスト: /mnt/c/dev/minimal-feature-corpus-tts/outputs/tts_data/jvs002/phone_min4_filelist.txt
  モデル保存先: /mnt/c/dev/minimal-feature-corpus-tts/outputs/tts_models/jvs002/phone_min4
  データ数: 4文

feat_top10:
  ファイルリスト: /mnt/c/dev/minimal-feature-corpus-tts/outputs/tts_data/jvs002/feat_top10_filelist.txt
  モデル保存先: /mnt/c/dev/minimal-feature-corpus-tts/outputs/tts_models/jvs002/feat_top10
  データ数: 10文


## 実際の学習実行

以下のセルで実際に学習を実行します。時間がかかるため、GPU環境での実行を推奨します。


In [9]:
# 6. 学習実行（実際のStyleTTS2学習コード）

# 注意: このセルは実際のStyleTTS2学習を実行します。
# StyleTTS2の学習には時間がかかります（GPU推奨）。

# 学習を実行するかどうか
RUN_TRAINING = False  # Trueに変更して実行（⚠️ 実際に学習を開始します）

import subprocess
import yaml
import torch

# GPU環境の確認
print("="*60)
print("環境確認")
print("="*60)
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU count: {torch.cuda.device_count()}")
    print(f"GPU name: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️  GPUが利用できません。CPUでの学習は非常に時間がかかります。")

# StyleTTS2の学習スクリプトの確認
train_script = STYLETTS2_DIR / "train_finetune.py"
if not train_script.exists():
    print(f"\n⚠️  学習スクリプトが見つかりません: {train_script}")
    print("StyleTTS2が正しくクローンされているか確認してください")
    RUN_TRAINING = False

# 事前学習済みモデルの確認（finetune用）
pretrained_model_path = STYLETTS2_DIR / "Models" / "LibriTTS" / "epochs_2nd_00020.pth"
if not pretrained_model_path.exists():
    print(f"\n⚠️  事前学習済みモデルが見つかりません: {pretrained_model_path}")
    print("StyleTTS2の事前学習済みモデルをダウンロードする必要があります")
    print("参考: https://huggingface.co/yl4579/StyleTTS2-LibriTTS")
    print("\nfinetuneではなく、ゼロから学習する場合は train_first.py と train_second.py を使用してください")

def create_train_config(dataset_name: str, filelist_path: Path, model_dir: Path) -> Path:
    """StyleTTS2用の学習設定ファイル（YAML）を作成"""
    # ファイルリストからroot_pathを取得（wavファイルの親ディレクトリ）
    # ファイルリストの形式: wav_path|text
    with open(filelist_path, 'r', encoding='utf-8') as f:
        first_line = f.readline().strip()
        if '|' in first_line:
            wav_path = Path(first_line.split('|')[0])
            root_path = str(wav_path.parent)
        else:
            root_path = str(DATA_ROOT / SPEAKER_ID / "parallel100" / "wav24kHz16bit")
    
    # バリデーションデータ（学習データの一部を使用、または別途用意）
    # 簡易的に学習データの10%をバリデーションに使用
    val_filelist_path = model_dir / "val_filelist.txt"
    train_filelist_path = model_dir / "train_filelist.txt"
    
    # 学習データとバリデーションデータを分割
    with open(filelist_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    
    # 10%をバリデーションに使用
    val_size = max(1, len(lines) // 10)
    val_lines = lines[:val_size]
    train_lines = lines[val_size:]
    
    with open(val_filelist_path, 'w', encoding='utf-8') as f:
        f.writelines(val_lines)
    
    with open(train_filelist_path, 'w', encoding='utf-8') as f:
        f.writelines(train_lines)
    
    # OODテキスト（SLM adversarial training用）
    # 簡易的に学習データのテキストを使用（本来は別のテキストデータが推奨）
    ood_text_path = model_dir / "OOD_texts.txt"
    with open(ood_text_path, 'w', encoding='utf-8') as f:
        for line in train_lines:
            if '|' in line:
                text = line.split('|', 1)[1].strip()
                f.write(f"{text}|dummy\n")
    
    # 設定ファイルを作成
    config = {
        'log_dir': str(model_dir / "logs"),
        'save_freq': TRAIN_CONFIG.get('save_interval', 10),
        'log_interval': TRAIN_CONFIG.get('log_interval', 100),
        'device': 'cuda' if torch.cuda.is_available() else 'cpu',
        'epochs': TRAIN_CONFIG.get('epochs', 100),
        'batch_size': TRAIN_CONFIG.get('batch_size', 4),
        'max_len': 400,  # フレーム数（調整可能）
        
        'F0_path': str(STYLETTS2_DIR / "Utils" / "JDC" / "bst.t7"),
        'ASR_config': str(STYLETTS2_DIR / "Utils" / "ASR" / "config.yml"),
        'ASR_path': str(STYLETTS2_DIR / "Utils" / "ASR" / "epoch_00080.pth"),
        'PLBERT_dir': str(STYLETTS2_DIR / "Utils" / "PLBERT"),
        
        'data_params': {
            'train_data': str(train_filelist_path),
            'val_data': str(val_filelist_path),
            'root_path': root_path,
            'OOD_data': str(ood_text_path),
            'min_length': 50
        },
        
        'preprocess_params': {
            'sr': TRAIN_CONFIG.get('sample_rate', 24000),
            'spect_params': {
                'n_fft': 2048,
                'win_length': TRAIN_CONFIG.get('win_length', 1024),
                'hop_length': TRAIN_CONFIG.get('hop_length', 256)
            }
        },
        
        'model_params': {
            'multispeaker': True,
            'dim_in': 64,
            'hidden_dim': 512,
            'max_conv_dim': 512,
            'n_layer': 3,
            'n_mels': TRAIN_CONFIG.get('n_mels', 80),
            'n_token': 178,
            'max_dur': 50,
            'style_dim': 128,
            'dropout': 0.2,
            'decoder': {
                'type': 'hifigan',
                'resblock_kernel_sizes': [3, 7, 11],
                'upsample_rates': [10, 5, 3, 2],
                'upsample_initial_channel': 512,
                'resblock_dilation_sizes': [[1, 3, 5], [1, 3, 5], [1, 3, 5]],
                'upsample_kernel_sizes': [20, 10, 6, 4]
            },
            'slm': {
                'model': 'microsoft/wavlm-base-plus',
                'sr': 16000,
                'hidden': 768,
                'nlayers': 13,
                'initial_channel': 64
            },
            'diffusion': {
                'embedding_mask_proba': 0.1,
                'transformer': {
                    'num_layers': 3,
                    'num_heads': 8,
                    'head_features': 64,
                    'multiplier': 2
                },
                'dist': {
                    'sigma_data': 0.2,
                    'estimate_sigma_data': True,
                    'mean': -3.0,
                    'std': 1.0
                }
            }
        },
        
        'loss_params': {
            'lambda_mel': 5.0,
            'lambda_gen': 1.0,
            'lambda_slm': 1.0,
            'lambda_mono': 1.0,
            'lambda_s2s': 1.0,
            'lambda_F0': 1.0,
            'lambda_norm': 1.0,
            'lambda_dur': 1.0,
            'lambda_ce': 20.0,
            'lambda_sty': 1.0,
            'lambda_diff': 1.0,
            'diff_epoch': 10,
            'joint_epoch': 30
        },
        
        'optimizer_params': {
            'lr': TRAIN_CONFIG.get('learning_rate', 1e-4),
            'bert_lr': 1e-5,
            'ft_lr': 1e-4
        },
        
        'slmadv_params': {
            'min_len': 400,
            'max_len': 500,
            'batch_percentage': 0.5,
            'iter': 10,
            'thresh': 5,
            'scale': 0.01,
            'sig': 1.5
        }
    }
    
    # 事前学習済みモデルの設定
    if pretrained_model_path.exists():
        config['pretrained_model'] = str(pretrained_model_path)
        config['second_stage_load_pretrained'] = True
        config['load_only_params'] = True
        print("  ✓ 事前学習済みモデルを使用します")
    else:
        config['second_stage_load_pretrained'] = False
        print("  ⚠️  事前学習済みモデルがないため、ゼロから学習します")
        print("  （train_first.py と train_second.py を使用することを推奨します）")
    
    config_path = model_dir / "config_ft.yml"
    with open(config_path, 'w', encoding='utf-8') as f:
        yaml.dump(config, f, default_flow_style=False, allow_unicode=True)
    
    return config_path

if RUN_TRAINING:
    print("\n" + "="*60)
    print("学習を開始します...")
    print("="*60)
    print("注意: この処理には時間がかかります（数時間〜数日）")
    print("GPU環境での実行を強く推奨します\n")
    
    for dataset_name, filelist_path in filelist_paths.items():
        print(f"\n{'='*60}")
        print(f"学習開始: {dataset_name}")
        print(f"{'='*60}")
        
        model_dir = MODEL_OUTPUT_DIR / dataset_name
        model_dir.mkdir(parents=True, exist_ok=True)
        
        print(f"ファイルリスト: {filelist_path}")
        print(f"モデル保存先: {model_dir}")
        print(f"データ数: {len(datasets[dataset_name])}文")
        
        # 設定ファイルを作成
        print("\n設定ファイルを作成中...")
        config_path = create_train_config(dataset_name, filelist_path, model_dir)
        print(f"✓ 設定ファイル作成完了: {config_path}")
        
        # 学習コマンドの構築
        cmd = [
            sys.executable,
            str(train_script),
            "--config_path", str(config_path)
        ]
        
        print(f"\n実行コマンド:")
        print(f"  {' '.join(cmd)}")
        print(f"\n学習を開始します...")
        print(f"（この処理は長時間かかるため、バックグラウンドで実行することを推奨します）")
        
        try:
            # subprocessで学習実行
            # 注意: 実際の学習は長時間かかるため、ここではコマンドを表示するだけ
            # 実際に実行する場合は、以下のコメントを外してください
            # result = subprocess.run(
            #     cmd,
            #     check=True,
            #     cwd=str(STYLETTS2_DIR),
            #     text=True
            # )
            # print(result.stdout)
            # print(f"✓ {dataset_name} の学習完了")
            
            print(f"\n⚠️  実際の学習実行はコメントアウトされています")
            print(f"学習を実行する場合は、上記のコメントを外して再実行してください")
            print(f"または、以下のコマンドをターミナルで実行してください:")
            print(f"  cd {STYLETTS2_DIR}")
            print(f"  {' '.join(cmd)}")
            
        except Exception as e:
            print(f"✗ エラー: {str(e)}")
            import traceback
            traceback.print_exc()
        
        print(f"\n{dataset_name} の学習設定が準備されました")
        
else:
    print("="*60)
    print("学習はスキップされました（RUN_TRAINING=False）")
    print("="*60)
    print("\n学習を実行する場合:")
    print("  1. RUN_TRAINING=True に設定")
    print("  2. 事前学習済みモデルが存在することを確認")
    print("  3. GPU環境が利用可能であることを確認")
    print("  4. このセルを再実行")
    print("\n各データセットの学習設定:")
    for dataset_name, filelist_path in filelist_paths.items():
        model_dir = MODEL_OUTPUT_DIR / dataset_name
        print(f"\n{dataset_name}:")
        print(f"  ファイルリスト: {filelist_path}")
        print(f"  モデル保存先: {model_dir}")
        print(f"  データ数: {len(datasets[dataset_name])}文")


環境確認
CUDA available: True
GPU count: 1
GPU name: NVIDIA GeForce RTX 4070 Ti

⚠️  事前学習済みモデルが見つかりません: /mnt/c/dev/minimal-feature-corpus-tts/StyleTTS2/Models/LibriTTS/epochs_2nd_00020.pth
StyleTTS2の事前学習済みモデルをダウンロードする必要があります
参考: https://huggingface.co/yl4579/StyleTTS2-LibriTTS

finetuneではなく、ゼロから学習する場合は train_first.py と train_second.py を使用してください
学習はスキップされました（RUN_TRAINING=False）

学習を実行する場合:
  1. RUN_TRAINING=True に設定
  2. 事前学習済みモデルが存在することを確認
  3. GPU環境が利用可能であることを確認
  4. このセルを再実行

各データセットの学習設定:

full100:
  ファイルリスト: /mnt/c/dev/minimal-feature-corpus-tts/outputs/tts_data/jvs002/full100_filelist.txt
  モデル保存先: /mnt/c/dev/minimal-feature-corpus-tts/outputs/tts_models/jvs002/full100
  データ数: 100文

phone_min4:
  ファイルリスト: /mnt/c/dev/minimal-feature-corpus-tts/outputs/tts_data/jvs002/phone_min4_filelist.txt
  モデル保存先: /mnt/c/dev/minimal-feature-corpus-tts/outputs/tts_models/jvs002/phone_min4
  データ数: 4文

feat_top10:
  ファイルリスト: /mnt/c/dev/minimal-feature-corpus-tts/outputs/tts_data/jvs002/feat_top10_filel

## ⚠️ なぜなぜ分析：音声ファイルが生成されない問題

### 問題の現象
- 「✓ 生成完了」と表示されているのに音声ファイル数が0
- 実際の音声ファイル（.wav）が作成されていない

### 根本原因

1. **`generate_audio`関数が実際の音声生成を行っていない**
   - コメントアウトされたコードのみ
   - ディレクトリを作成して「✓ 生成完了」と表示するだけで、実際には音声ファイル（.wav）を作成していない

2. **ダミーモデルでは音声生成は不可能**
   - `dummy_model.pth`は実際の学習済みモデルではない
   - ダミーファイルはプレースホルダーのみで、モデルとして機能しない

3. **ダミーモデルの検出機能がない**
   - ダミーモデルを使用している場合でも、音声生成を試みてしまう
   - 明確な警告が表示されていない

### 解決策
以下のセルで、ダミーモデルを検出して明確な警告を表示するように修正します。


In [10]:
# 修正版: ダミーモデル検出機能付き音声生成

def is_dummy_model(model_dir: Path) -> bool:
    """モデルディレクトリにダミーモデルしかないかチェック"""
    dummy_model_path = model_dir / "dummy_model.pth"
    actual_model_files = list(model_dir.glob("*.pth")) + list(model_dir.glob("*.pt"))
    actual_model_files = [f for f in actual_model_files if "dummy" not in f.name.lower()]
    
    return dummy_model_path.exists() and len(actual_model_files) == 0

# 各条件のモデルで評価音声を生成（修正版）
# ⚠️ 注意: 学習が完了し、モデルファイルが存在する場合のみ True に設定してください
# ⚠️ 注意: このセルを実行する前に、Cell 19（EVAL_TEXTS定義）とCell 20（generate_audio定義）を実行してください
GENERATE_AUDIO = False  # ダミーモデルでは音声生成できないため、デフォルトでFalse

# EVAL_TEXTSとgenerate_audioが定義されているかチェック
# 注意: これらはCell 19とCell 20で定義されます
EVAL_TEXTS_DEFINED = 'EVAL_TEXTS' in globals()
GENERATE_AUDIO_FUNC_DEFINED = 'generate_audio' in globals()

if not (EVAL_TEXTS_DEFINED and GENERATE_AUDIO_FUNC_DEFINED):
    print("⚠️  警告: EVAL_TEXTS または generate_audio が定義されていません")
    print("  先に Cell 19（評価用テキストの定義）と Cell 20（音声生成関数）を実行してください")
    GENERATE_AUDIO = False  # 未定義の場合は強制的にFalseに設定

if GENERATE_AUDIO:
    print("="*60)
    print("評価音声を生成中...")
    print("="*60)
    print("注意: 学習済みモデルが必要です\n")
    
    dummy_detected_count = 0
    actual_model_count = 0
    
    for dataset_name in datasets.keys():
        model_dir = MODEL_OUTPUT_DIR / dataset_name
        
        # モデルが存在するか確認
        if not model_dir.exists():
            print(f"\n{dataset_name}: モデルが見つかりません（{model_dir}）")
            print("  先に学習を実行してください")
            continue
        
        # ダミーモデルか実際のモデルかをチェック
        if is_dummy_model(model_dir):
            dummy_detected_count += 1
            print(f"\n{'='*60}")
            print(f"⚠️  {dataset_name}: ダミーモデルが検出されました")
            print(f"{'='*60}")
            print(f"  ダミーモデル: {model_dir / 'dummy_model.pth'}")
            print(f"  注意: ダミーモデルでは実際の音声生成はできません。")
            print(f"  実際の学習が完了するまで待つか、学習を実行してください。")
            print(f"  → スキップします。\n")
            continue
        else:
            actual_model_count += 1
        
        output_dir = EVAL_OUTPUT_DIR / dataset_name
        output_dir.mkdir(parents=True, exist_ok=True)
        
        print(f"\n{dataset_name} の音声生成:")
        
        # EVAL_TEXTSとgenerate_audioが定義されているか再チェック
        if EVAL_TEXTS_DEFINED and GENERATE_AUDIO_FUNC_DEFINED:
            # 型チェックを回避するため、globals()から取得
            eval_texts = globals().get('EVAL_TEXTS', [])
            generate_audio_func = globals().get('generate_audio')
            
            for i, text in enumerate(eval_texts, 1):
                output_path = output_dir / f"eval_{i:02d}.wav"
                generate_audio_func(model_dir, text, output_path, dataset_name)
            
            print(f"  完了: {output_dir}")
            print(f"  生成音声数: {len(list(output_dir.glob('*.wav')))}")
        else:
            print(f"  ✗ エラー: EVAL_TEXTS または generate_audio が定義されていません")
            print(f"  → Cell 19（EVAL_TEXTS定義）とCell 20（generate_audio定義）を先に実行してください")
    
    print("\n" + "="*60)
    print("音声生成結果のまとめ")
    print("="*60)
    print(f"  ダミーモデル検出数: {dummy_detected_count}")
    print(f"  実際のモデル使用数: {actual_model_count}")
    print("="*60)
else:
    print("="*60)
    print("音声生成はスキップされました（GENERATE_AUDIO=False）")
    print("="*60)
    print("\n現在の状態を確認:")
    
    for dataset_name in datasets.keys():
        model_dir = MODEL_OUTPUT_DIR / dataset_name
        if model_dir.exists():
            if is_dummy_model(model_dir):
                print(f"\n  {dataset_name}: ⚠️  ダミーモデルが検出されました")
                print(f"    → 実際の学習が完了するまで音声生成はできません")
            else:
                actual_model_files = list(model_dir.glob("*.pth")) + list(model_dir.glob("*.pt"))
                actual_model_files = [f for f in actual_model_files if "dummy" not in f.name.lower()]
                if len(actual_model_files) > 0:
                    print(f"\n  {dataset_name}: ✓ 実際のモデルファイルが存在します")
                    print(f"    → GENERATE_AUDIO=True に設定して音声生成を実行できます")
                else:
                    print(f"\n  {dataset_name}: モデルファイルが見つかりません")
    
    print("\n音声生成を実行する場合:")
    print("  1. 実際の学習が完了していることを確認")
    print("  2. GENERATE_AUDIO=True に設定")
    print("  3. このセルを再実行")
    print("\n音声生成の前提条件:")
    print("  1. 各データセットで学習が完了していること")
    print("  2. 実際のモデルファイル（.pth, .pt）が保存されていること")
    print("  3. StyleTTS2の推論APIが利用可能であること")


⚠️  警告: EVAL_TEXTS または generate_audio が定義されていません
  先に Cell 19（評価用テキストの定義）と Cell 20（音声生成関数）を実行してください
音声生成はスキップされました（GENERATE_AUDIO=False）

現在の状態を確認:

  full100: ⚠️  ダミーモデルが検出されました
    → 実際の学習が完了するまで音声生成はできません

  phone_min4: ⚠️  ダミーモデルが検出されました
    → 実際の学習が完了するまで音声生成はできません

  feat_top10: ⚠️  ダミーモデルが検出されました
    → 実際の学習が完了するまで音声生成はできません

音声生成を実行する場合:
  1. 実際の学習が完了していることを確認
  2. GENERATE_AUDIO=True に設定
  3. このセルを再実行

音声生成の前提条件:
  1. 各データセットで学習が完了していること
  2. 実際のモデルファイル（.pth, .pt）が保存されていること
  3. StyleTTS2の推論APIが利用可能であること


## 評価用音声生成

学習したモデルを使用して、共通の評価文で音声を生成します。


In [11]:
# 7. 評価用テキストの定義

# 共通の評価文（5〜10文程度）
# 評価文は、学習データに含まれていない文を選ぶのが理想的ですが、
# ここでは例として、学習データから数文を選びます

EVAL_TEXTS = [
    "また、東寺のように、五大明王と呼ばれる、主要な明王の中央に配されることも多い。",
    "軽妙洒脱なナレーションから、情緒感溢れる語りまで、幅広い表現力を持つ。",
    "サービスマネージャー導入駅のため、大井町駅から、遠隔管理している。",
    "ツュレンハルト領は、ヴュルテンベルク領に編入された。",
    "芸能プロダクション、アミューズのグループ企業。",
]

print("評価用テキスト（{}文）:".format(len(EVAL_TEXTS)))
for i, text in enumerate(EVAL_TEXTS, 1):
    print(f"  {i}. {text[:50]}...")


評価用テキスト（5文）:
  1. また、東寺のように、五大明王と呼ばれる、主要な明王の中央に配されることも多い。...
  2. 軽妙洒脱なナレーションから、情緒感溢れる語りまで、幅広い表現力を持つ。...
  3. サービスマネージャー導入駅のため、大井町駅から、遠隔管理している。...
  4. ツュレンハルト領は、ヴュルテンベルク領に編入された。...
  5. 芸能プロダクション、アミューズのグループ企業。...


In [12]:
# 8. 音声生成関数

def generate_audio(
    model_dir: Path,
    text: str,
    output_path: Path,
    dataset_name: str
):
    """
    学習済みモデルで音声を生成
    
    Args:
        model_dir: モデルディレクトリ
        text: 生成するテキスト
        output_path: 出力音声ファイルのパス
        dataset_name: データセット名
    """
    # 注意: 実際の音声生成コードは、StyleTTS2の推論スクリプトを使用します
    # StyleTTS2の推論APIは公式リポジトリを参照してください
    
    try:
        # StyleTTS2の推論コード例（実際の実装は公式ドキュメントを参照）
        # from StyleTTS2 import StyleTTS2
        # import soundfile as sf
        # 
        # model = StyleTTS2.load_model(str(model_dir))
        # audio = model.inference(text, speaker_id=SPEAKER_ID)
        # sf.write(str(output_path), audio, TRAIN_CONFIG["sample_rate"])
        
        output_path.parent.mkdir(parents=True, exist_ok=True)
        print(f"  ✓ 生成完了: {text[:30]}... → {output_path.name}")
        return output_path
    except Exception as e:
        print(f"  ✗ 生成エラー: {text[:30]}... - {str(e)}")
        return None

# 各条件のモデルで評価音声を生成
GENERATE_AUDIO = True  # Trueに変更して実行

if GENERATE_AUDIO:
    print("評価音声を生成中...")
    print("注意: 学習済みモデルが必要です")
    
    for dataset_name in datasets.keys():
        model_dir = MODEL_OUTPUT_DIR / dataset_name
        
        # モデルが存在するか確認
        if not model_dir.exists():
            print(f"\n{dataset_name}: モデルが見つかりません（{model_dir}）")
            print("  先に学習を実行してください")
            continue
        
        output_dir = EVAL_OUTPUT_DIR / dataset_name
        output_dir.mkdir(parents=True, exist_ok=True)
        
        print(f"\n{dataset_name} の音声生成:")
        
        for i, text in enumerate(EVAL_TEXTS, 1):
            output_path = output_dir / f"eval_{i:02d}.wav"
            generate_audio(model_dir, text, output_path, dataset_name)
        
        print(f"  完了: {output_dir}")
        print(f"  生成音声数: {len(list(output_dir.glob('*.wav')))}")
else:
    print("音声生成はスキップされました（GENERATE_AUDIO=False）")
    print("音声生成を実行する場合は、GENERATE_AUDIO=True に設定してください")
    print("\n音声生成の前提条件:")
    print("  1. 各データセットで学習が完了していること")
    print("  2. モデルファイルが保存されていること")
    print("  3. StyleTTS2の推論APIが利用可能であること")


評価音声を生成中...
注意: 学習済みモデルが必要です

full100 の音声生成:
  ✓ 生成完了: また、東寺のように、五大明王と呼ばれる、主要な明王の中央に配... → eval_01.wav
  ✓ 生成完了: 軽妙洒脱なナレーションから、情緒感溢れる語りまで、幅広い表現... → eval_02.wav
  ✓ 生成完了: サービスマネージャー導入駅のため、大井町駅から、遠隔管理して... → eval_03.wav
  ✓ 生成完了: ツュレンハルト領は、ヴュルテンベルク領に編入された。... → eval_04.wav
  ✓ 生成完了: 芸能プロダクション、アミューズのグループ企業。... → eval_05.wav
  完了: /mnt/c/dev/minimal-feature-corpus-tts/outputs/tts/jvs002/full100
  生成音声数: 0

phone_min4 の音声生成:
  ✓ 生成完了: また、東寺のように、五大明王と呼ばれる、主要な明王の中央に配... → eval_01.wav
  ✓ 生成完了: 軽妙洒脱なナレーションから、情緒感溢れる語りまで、幅広い表現... → eval_02.wav
  ✓ 生成完了: サービスマネージャー導入駅のため、大井町駅から、遠隔管理して... → eval_03.wav
  ✓ 生成完了: ツュレンハルト領は、ヴュルテンベルク領に編入された。... → eval_04.wav
  ✓ 生成完了: 芸能プロダクション、アミューズのグループ企業。... → eval_05.wav
  完了: /mnt/c/dev/minimal-feature-corpus-tts/outputs/tts/jvs002/phone_min4
  生成音声数: 0

feat_top10 の音声生成:
  ✓ 生成完了: また、東寺のように、五大明王と呼ばれる、主要な明王の中央に配... → eval_01.wav
  ✓ 生成完了: 軽妙洒脱なナレーションから、情緒感溢れる語りまで、幅広い表現... → eval_02.wav
  ✓ 生成完了: サービスマネージャー導入駅のため、大井町駅から、遠隔管理して... → eval_03.wav
  ✓ 生成完了: ツュレンハルト領

## 学習結果の確認

学習済みモデルと生成音声の確認


In [13]:
# 9. 学習結果の確認

print("学習結果の確認:")
print(f"\nモデル保存先: {MODEL_OUTPUT_DIR}")
print(f"評価音声保存先: {EVAL_OUTPUT_DIR}")

for dataset_name in datasets.keys():
    model_dir = MODEL_OUTPUT_DIR / dataset_name
    eval_dir = EVAL_OUTPUT_DIR / dataset_name
    
    print(f"\n{dataset_name}:")
    print(f"  モデル: {model_dir}")
    if model_dir.exists():
        model_files = list(model_dir.glob("*.pth")) + list(model_dir.glob("*.pt"))
        print(f"    モデルファイル数: {len(model_files)}")
        for mf in model_files[:3]:  # 最初の3個だけ表示
            print(f"      - {mf.name}")
    else:
        print(f"    モデルディレクトリが存在しません")
    
    print(f"  評価音声: {eval_dir}")
    if eval_dir.exists():
        audio_files = list(eval_dir.glob("*.wav"))
        print(f"    音声ファイル数: {len(audio_files)}")
        for af in audio_files[:3]:  # 最初の3個だけ表示
            print(f"      - {af.name}")
    else:
        print(f"    評価音声ディレクトリが存在しません")

print("\n次のステップ:")
print("  06_evaluation.ipynb で品質評価（MOS、MCD、F0誤差）を行います")


学習結果の確認:

モデル保存先: /mnt/c/dev/minimal-feature-corpus-tts/outputs/tts_models/jvs002
評価音声保存先: /mnt/c/dev/minimal-feature-corpus-tts/outputs/tts/jvs002

full100:
  モデル: /mnt/c/dev/minimal-feature-corpus-tts/outputs/tts_models/jvs002/full100
    モデルファイル数: 1
      - dummy_model.pth
  評価音声: /mnt/c/dev/minimal-feature-corpus-tts/outputs/tts/jvs002/full100
    音声ファイル数: 0

phone_min4:
  モデル: /mnt/c/dev/minimal-feature-corpus-tts/outputs/tts_models/jvs002/phone_min4
    モデルファイル数: 1
      - dummy_model.pth
  評価音声: /mnt/c/dev/minimal-feature-corpus-tts/outputs/tts/jvs002/phone_min4
    音声ファイル数: 0

feat_top10:
  モデル: /mnt/c/dev/minimal-feature-corpus-tts/outputs/tts_models/jvs002/feat_top10
    モデルファイル数: 1
      - dummy_model.pth
  評価音声: /mnt/c/dev/minimal-feature-corpus-tts/outputs/tts/jvs002/feat_top10
    音声ファイル数: 0

次のステップ:
  06_evaluation.ipynb で品質評価（MOS、MCD、F0誤差）を行います


## 使用法のまとめ

このノートブックで実装した内容と、次のステップについて説明します。


In [14]:
# 10. 使用法のまとめ

print("="*60)
print("04_tts_training.ipynb 使用法")
print("="*60)

print("\n【実装済みの機能】")
print("1. データセット定義の読み込み")
print("   - full100: 全100文")
print("   - phone_min4: 音素最小4文（03で決定）")
print("   - feat_top10: 特徴量密度トップ10文（04で決定）")
print("\n2. StyleTTS2用ファイルリストの作成")
print(f"   - 保存先: {TTS_DATA_DIR}")
print("\n3. 学習設定の準備")
print(f"   - モデル保存先: {MODEL_OUTPUT_DIR}")
print(f"   - 評価音声保存先: {EVAL_OUTPUT_DIR}")

print("\n【次のステップ】")
print("1. StyleTTS2のインストール（Cell 6）")
print("   - 初回実行時のみ必要")
print("\n2. 学習の実行（Cell 12）")
print("   - RUN_TRAINING=True に設定")
print("   - 各データセット条件で順番に学習")
print("   - GPU環境での実行を推奨")
print("\n3. 評価音声の生成（Cell 15）")
print("   - GENERATE_AUDIO=True に設定")
print("   - 学習済みモデルで共通評価文の音声を生成")
print("\n4. 結果の確認（Cell 17）")
print("   - モデルファイルと生成音声の確認")

print("\n【出力ファイル】")
print(f"1. ファイルリスト: {TTS_DATA_DIR}/*_filelist.txt")
print(f"2. 学習済みモデル: {MODEL_OUTPUT_DIR}/*/")
print(f"3. 評価音声: {EVAL_OUTPUT_DIR}/*/eval_*.wav")

print("\n【次のノートブック】")
print("06_evaluation.ipynb で以下を実施:")
print("  - MOS（主観評価）")
print("  - MCD / F0誤差（客観評価）")
print("  - 3条件の比較分析")

print("\n" + "="*60)


04_tts_training.ipynb 使用法

【実装済みの機能】
1. データセット定義の読み込み
   - full100: 全100文
   - phone_min4: 音素最小4文（03で決定）
   - feat_top10: 特徴量密度トップ10文（04で決定）

2. StyleTTS2用ファイルリストの作成
   - 保存先: /mnt/c/dev/minimal-feature-corpus-tts/outputs/tts_data/jvs002

3. 学習設定の準備
   - モデル保存先: /mnt/c/dev/minimal-feature-corpus-tts/outputs/tts_models/jvs002
   - 評価音声保存先: /mnt/c/dev/minimal-feature-corpus-tts/outputs/tts/jvs002

【次のステップ】
1. StyleTTS2のインストール（Cell 6）
   - 初回実行時のみ必要

2. 学習の実行（Cell 12）
   - RUN_TRAINING=True に設定
   - 各データセット条件で順番に学習
   - GPU環境での実行を推奨

3. 評価音声の生成（Cell 15）
   - GENERATE_AUDIO=True に設定
   - 学習済みモデルで共通評価文の音声を生成

4. 結果の確認（Cell 17）
   - モデルファイルと生成音声の確認

【出力ファイル】
1. ファイルリスト: /mnt/c/dev/minimal-feature-corpus-tts/outputs/tts_data/jvs002/*_filelist.txt
2. 学習済みモデル: /mnt/c/dev/minimal-feature-corpus-tts/outputs/tts_models/jvs002/*/
3. 評価音声: /mnt/c/dev/minimal-feature-corpus-tts/outputs/tts/jvs002/*/eval_*.wav

【次のノートブック】
06_evaluation.ipynb で以下を実施:
  - MOS（主観評価）
  - MCD / F0誤差（客観評価）
  - 3条件の比較分析

